In [1]:
import pandas as pd, ast
import numpy as np
import os
from tqdm import tqdm
import time
import ast
import pickle
import torch
from sentence_transformers import SentenceTransformer, util
import logging
from torch import Tensor, device
from collections import deque
from sklearn.cluster import KMeans
import logging
logger = logging.getLogger(__name__)
help(logger.info)

2024-02-18 11:25:40.002662: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-18 11:25:40.133710: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-18 11:25:40.746233: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:
2024-02-18 11:25:40.746325: W tensorflow/c

Help on method info in module logging:

info(msg, *args, **kwargs) method of logging.Logger instance
    Log 'msg % args' with severity 'INFO'.
    
    To pass exception information, use the keyword argument exc_info with
    a true value, e.g.
    
    logger.info("Houston, we have a %s", "interesting problem", exc_info=1)



In [2]:
class Data_load():
    def __init__(self):
        pass
    
    def data_processing(self):

        df =pd.read_csv(INPUT_FILE)
        df.drop_duplicates(INPUT_ID, inplace=True)
        df = df.loc[df[INPUT_COLUMN_1].isnull()== False]
        df.reset_index(drop=True, inplace=True)
        try:
            df[INPUT_COLUMN_1]=df[INPUT_COLUMN_1].apply(lambda x: ast.literal_eval(x))
        except:
            a=1

        try:
            df[INPUT_COLUMN_2]=df[INPUT_COLUMN_2].apply(lambda x: ast.literal_eval(x))
        except:
            a=1

        try:
            df[INPUT_COLUMN_3]=df[INPUT_COLUMN_3].apply(lambda x: ast.literal_eval(x))
        except:
            a=1

        return df
    
    def embeddings(self, final):

        sentencemodel = SentenceTransformer(SENTENCE_MODEL_PATH)
        col = list(range(EMBED_DIM)) + [INPUT_ID,'phrase_no']
        X1 = pd.DataFrame(columns = col)

        if final.shape[0] > 0:
            for i in tqdm(range(final.shape[0])):
                summary_sentences = final[INPUT_COLUMN_1].values[i]
                if i==0:
                    print('summary_sentences ', summary_sentences)
                new_phrases = []
                phrase_no = 0

                if type(summary_sentences)==list:

                    for c in range(len(summary_sentences)):
                        t = summary_sentences[c].strip()
                        phrase_embeddings = sentencemodel.encode(summary_sentences[c].strip(), device='cuda')
                        phrase_embeddings = torch.tensor(phrase_embeddings)
                        phrase_embeddings = phrase_embeddings.tolist()
                        list_add = phrase_embeddings + [final.loc[i,INPUT_ID],phrase_no]
                        phrase_no += 1
                        X1.loc[len(X1)] = list_add

        X1.to_pickle('./data_full_embeddings_smp.pickle')
        return X1


In [3]:
class CommunityDetection:
    def __init__(self):
        pass
    
    def cos_sim(self, a: Tensor, b: Tensor):

        if not isinstance(a, torch.Tensor):
            a = torch.tensor(a)

        if not isinstance(b, torch.Tensor):
            b = torch.tensor(b)

        if len(a.shape) == 1:
            a = a.unsqueeze(0)

        if len(b.shape) == 1:
            b = b.unsqueeze(0)

        a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
        b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
        return torch.mm(a_norm, b_norm.transpose(0, 1))
    
    def community_detection_modified_v2(self, embeddings, threshold, min_community_size, batch_size):
        """
        Function for Fast Community Detection
        Finds in the embeddings all communities, i.e. embeddings that are close (closer than threshold).
        Returns only communities that are larger than min_community_size. The communities are returned
        in decreasing order. The first element in each list is the central point in the community.
        """
        mapping_row_index=dict(zip(range(len(embeddings)),list(embeddings.index)))

        embeddings = torch.from_numpy(embeddings.to_numpy().astype('float64'))

        if not isinstance(embeddings, torch.Tensor):
            embeddings = torch.tensor(embeddings)

        threshold = torch.tensor(threshold, device=embeddings.device)
        extracted_communities = deque()

        # Maximum size for community
        min_community_size = min(min_community_size, len(embeddings))
        sort_max_size = min(max(4 * min_community_size, 50), len(embeddings))

        for start_idx in range(0, len(embeddings), batch_size):

            # Compute cosine similarity scores
            cos_scores = self.cos_sim(embeddings[start_idx:start_idx + batch_size], embeddings)

            # Minimum size for a community
            top_k_values, _ = cos_scores.topk(k=min_community_size, largest=True)

            # Filter for rows >= min_threshold
            for i in range(len(top_k_values)):

                if top_k_values[i][-1] >= threshold:
                    new_cluster = []

                    # Only check top k most similar entries
                    top_val_large, top_idx_large = cos_scores[i].topk(k=sort_max_size, largest=True)

                    # Check if we need to increase sort_max_size
                    while top_val_large[-1] > threshold and sort_max_size < len(embeddings):
                        sort_max_size = min(2 * sort_max_size, len(embeddings))
                        top_val_large, top_idx_large = cos_scores[i].topk(k=sort_max_size, largest=True)

                    for idx, val in zip(top_idx_large.tolist(), top_val_large):
                        if val < threshold:
                            break

                        new_cluster.append(idx)

                    extracted_communities.append(new_cluster)

            del cos_scores

        extracted_communities=list(extracted_communities)
        ## extracted_communities

        # Largest cluster first
        extracted_communities = sorted(extracted_communities, key=lambda x: len(x), reverse=True)

        # Step 2) Remove overlapping communities
        unique_communities = deque()
        extracted_ids = set()
        first_elem_list=deque()

        for cluster_id, community in enumerate(extracted_communities):

            non_overlapped_community = []      
            first_elem=community[0]

            if first_elem in extracted_ids:
                continue
            else:
                for idx in community:
                    if idx not in extracted_ids:
                        non_overlapped_community.append(idx)

            if len(non_overlapped_community) >= min_community_size:
                first_elem_list.append([first_elem])
                unique_communities.append(non_overlapped_community)
                extracted_ids.update(non_overlapped_community)

        unique_communities=list(unique_communities)
        first_elem_list=list(first_elem_list)

        unique_communities_upd=deque()
        
        for i, unique_comm in enumerate(unique_communities):

            comm=[mapping_row_index[j] for j in unique_comm]
            unique_communities_upd.append(comm)
            


        unique_communities=list(unique_communities_upd)
        return unique_communities
    

In [4]:

class Clustering(CommunityDetection):
    def __init__(self, df, X):
        super().__init__()
        
        df=df.loc[df[INPUT_COLUMN_1].isnull()==False]
        df.reset_index(drop=True, inplace=True)

        
        try:
            df[INPUT_COLUMN_1]=df[INPUT_COLUMN_1].apply(lambda x: ast.literal_eval(x))
        except:
            a=1

        try:
            df[INPUT_COLUMN_2]=df[INPUT_COLUMN_2].apply(lambda x: ast.literal_eval(x))
        except:
            a=1

        try:
            df[INPUT_COLUMN_3]=df[INPUT_COLUMN_3].apply(lambda x: ast.literal_eval(x))
        except:
            a=1
            
        df['phrase_no']=df[INPUT_COLUMN_1].apply(lambda x: list(range(len(x))) )
        
        start=time.time()
        df=df.explode([INPUT_COLUMN_1, INPUT_COLUMN_2, INPUT_COLUMN_3, 'phrase_no'])
        end=time.time()
        logger.info('Time taken in explode is %s %s ', (end-start), 'secs')

        start=time.time()
        df_info=df.loc[:, [INPUT_ID, INPUT_COLUMN_1, INPUT_COLUMN_2, INPUT_COLUMN_3,  'phrase_no', INPUT_TEXT]]
        df_X=pd.merge(df_info, X, how ='inner', on=[INPUT_ID, 'phrase_no'])
        df_X.reset_index(drop=True, inplace=True)
        
        long_description_list=df_X[INPUT_COLUMN_1].to_list()
        verbose_label_list=df_X[INPUT_COLUMN_2].to_list()
        succinct_label_list=df_X[INPUT_COLUMN_3].to_list()
        
        X=df_X.drop([INPUT_ID, INPUT_COLUMN_1, INPUT_COLUMN_2, INPUT_COLUMN_3,  'phrase_no', INPUT_TEXT], axis=1)
        df_raw=df_X.loc[:, [INPUT_ID,INPUT_COLUMN_1, INPUT_COLUMN_2, INPUT_COLUMN_3, INPUT_TEXT]]
    
        end=time.time()
        logger.info('Time taken in merge is %s %s', (end-start), 'secs')
        
        self.long_description_list=long_description_list
        self.verbose_label_list=verbose_label_list
        self.succinct_label_list=succinct_label_list
        self.X=X
        self.df_raw=df_raw
        self.df_X=df_X
        
    def adding_clusters(self, indices_cluster, cluster_level, corresponding_level, df_raw_nc):
        
        long_description_list_nc=self.long_description_list.copy()
        verbose_label_list_nc=self.verbose_label_list.copy()
        succinct_label_list_nc=self.succinct_label_list.copy()
        
        X_nc=self.X.copy()
        
        if cluster_level=='0':
            label="Cluster: " + str(corresponding_level)
        
        elif cluster_level=='1':
            index=indices_cluster[0]
            label=succinct_label_list_nc[index]
        
        elif cluster_level=='2':
            index=indices_cluster[0]
            label=verbose_label_list_nc[index]
        
        else:
            label=cluster_level
        
        #cluster_center_embeddings = X_nc.iloc[indices_cluster].mean().tolist()
        cluster_long_description_list=[long_description_list_nc[ii] for ii in indices_cluster]
        cluster_top_sentences=cluster_long_description_list[:min(3, len(cluster_long_description_list))]
        
        df_raw_nc.loc[indices_cluster, 'L'+cluster_level+'_cluster_no' ]=corresponding_level
        df_raw_nc.loc[indices_cluster, 'L'+cluster_level+'_label' ]=label
        
        #df_raw_nc.loc[indices_cluster, 'L'+cluster_level+'_cluster_center_Embeddings' ]=str(cluster_center_embeddings)
        df_raw_nc.loc[indices_cluster, 'L'+cluster_level+'_cluster_top_sentences' ]=str(cluster_top_sentences)
        
        return df_raw_nc

        
    def clustering(self,  L1_th, L2_th, min_L1, min_L2):
        
        X_f=self.X.copy()
        df_raw_f=self.df_raw.copy()
        
        kmeans_clusters = max(int(X_f.shape[0]/5000)+1,2)
    
        logger.info('X %s', X_f.shape)

        start_kmeans = time.time()
        kmeans = KMeans(n_clusters=kmeans_clusters,random_state=0).fit(X_f)
        end_kmeans = time.time()
        logger.info("TIME TAKEN KMEANS: %s",round(end_kmeans-start_kmeans,2))
        target = kmeans.labels_


        for k_searchval in range(kmeans_clusters):
            start_b = time.time()
            indices_L0 = [i for i in np.where(target == k_searchval)[0]]
            logger.info('kmeans size %s',len(indices_L0) )

            
            cluster_level='0'
            
            corresponding_level=k_searchval
            df_raw_f=self.adding_clusters(indices_L0, cluster_level, corresponding_level, df_raw_f )

            start = time.time()

            X_L0=X_f.iloc[indices_L0]
            clusters_L1=self.community_detection_modified_v2(X_L0, threshold=L1_th, min_community_size=min_L1, batch_size=X_f.shape[0] )

            aa1 = time.time()
            logger.info("At CD: %s", str(round(aa1-start,2)) )

            start_L1=time.time()            

            for cluster_no_L1, c_L1 in enumerate(clusters_L1):
                
                cluster_level='1'
                corresponding_level='L_'+str(k_searchval)+'_'+str(cluster_no_L1)
                
                df_raw_f=self.adding_clusters(c_L1, cluster_level, corresponding_level, df_raw_f)    
                
                X_L1 = X_f.iloc[c_L1]
                clusters_L2 = self.community_detection_modified_v2(X_L1, threshold=L2_th, min_community_size=min_L2, batch_size=X_L1.shape[0] )

                for cluster_no_L2, c_L2 in enumerate(clusters_L2):
                    
                    cluster_level='2'
                    corresponding_level='L_'+str(k_searchval)+'_'+str(cluster_no_L1)+'_'+str(cluster_no_L2)
                    df_raw_f=self.adding_clusters(c_L2, cluster_level, corresponding_level, df_raw_f)     

            end_L1=time.time()

            logger.info('Time taken in L1 is %s %s', (end_L1-start_L1), ' secs')
        
        df_raw_f['K_val']=df_raw_f['L0_cluster_no']
         
        return df_raw_f
    
   

In [5]:
#Clustering metric
from sentence_transformers import SentenceTransformer, util
import torch
from torch import Tensor, device
import time
from sklearn import metrics
import numpy as np

class Clustering_metric():
    
    def __init__(self):
        
        
        SENTENCE_MODEL_PATH = '/data2/bdlml/mkabra/pretrained_model/bge-large-en-v1.5' ## Path to Sentence Transformer Model
        self.sentencemodel_bge = SentenceTransformer(SENTENCE_MODEL_PATH)

        pass
    
    def cos_sim(self, a: Tensor, b: Tensor):

        if not isinstance(a, torch.Tensor):
            a = torch.tensor(a)

        if not isinstance(b, torch.Tensor):
            b = torch.tensor(b)

        if len(a.shape) == 1:
            a = a.unsqueeze(0)

        if len(b.shape) == 1:
            b = b.unsqueeze(0)

        a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
        b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
        return torch.mm(a_norm, b_norm.transpose(0, 1))
    
    def cohesion_computation(self, cohesion_input ):
        cohesion_calc=0
        m=0
        s=time.time()
        for i in range(len(cohesion_input)):    
            sent1_list=[cohesion_input[i]]
            sent2_list=[]

            for j in range(i+1, len(cohesion_input)):
                sent2_list.append(cohesion_input[j])

            if len(sent2_list)>=1:
                sent1_embedding= self.sentencemodel_bge.encode(sent1_list)
                sent2_embedding= self.sentencemodel_bge.encode(sent2_list)
                sim=(self.cos_sim(sent1_embedding, sent2_embedding )).detach().numpy()[0]

                #print(sim, sim.sum(), len(sim))
                cohesion_calc+=sim.sum()
                m+=len(sim)
        e=time.time()
        cohesion_calc=cohesion_calc/m
        #print('cohesion_calc ', cohesion_calc)
        #print('cohesion block ', (e-s), 'secs')
        return cohesion_calc
    
    def topic_cohesion(self, df, pred_col, pred_col_label, input_var, k):
        df_clustered=df.loc[:, [pred_col, pred_col_label, input_var]].groupby([pred_col, pred_col_label]).agg(tuple).applymap(list).reset_index()
        pred_col_list=df_clustered[pred_col].to_list()
        
        all_clusters_cohesion=0
        
        
        for n, col in enumerate(pred_col_list):
            s=time.time()
            
                
            label=df_clustered.loc[df_clustered[pred_col]==col][pred_col_label].to_numpy()
            input_var_arr=df_clustered.loc[df_clustered[pred_col]==col][[input_var]].to_numpy()[0][0]
            
            
            try:
                label_embeddings = self.sentencemodel_bge.encode(label)
            except Exception as e:
                print('Block label')
                print(e)
                print(n, col)
                print(label)
                print('\n')
            
            try:
                input_label_embeddings = self.sentencemodel_bge.encode(input_var_arr)
            except Exception as e:
                print('Block input_var_arr')
                print(e)
                print(n, col)
                print(input_var_arr)
                print('\n')
                
            cos_scores=self.cos_sim(label_embeddings,input_label_embeddings )
            
           
            top_k_values, indices = cos_scores.topk(k=min(k, len(input_label_embeddings)) , largest=True)
           
            cohesion_input=[]
            for i in indices[0]:
                cohesion_input.append(input_var_arr[i])

            e=time.time()
            
            if len(cohesion_input)==1:
                print('cohesion_input', cohesion_input, 'col ', col)
            cluster_cohesion=self.cohesion_computation(cohesion_input)
            
            all_clusters_cohesion+=cluster_cohesion
            
            e2=time.time()
            
        n+=1
        if n>0:
            cohesion=all_clusters_cohesion/n
        else:
            cohesion='nis0'
        
        return cohesion
   

    def calculate_purity(self, true_col, pred_col, df): 
        '''
        Calculate harmonic purity between two set of clusterings
        df: a Pandas data frame containing two columns (true_col and pred_col)
        true_col: column containing a ground-truth label for each document 
        pred_col: column containing a predicted label for each document
        '''
        contingency_matrix = metrics.cluster.contingency_matrix(df[true_col], df[pred_col]) 
        precision = contingency_matrix / contingency_matrix.sum(axis=0).reshape(1, -1)
        recall = contingency_matrix / contingency_matrix.sum(axis=1).reshape(-1, 1)
        f1 = 2 * (precision * recall) / (precision + recall)
        f1 = np.nan_to_num(f1)
        purity = (np.amax(precision, axis=0) * contingency_matrix.sum(axis=0)).sum() / contingency_matrix.sum()
        inverse_purity = (np.amax(recall, axis=1) * contingency_matrix.sum(axis=1)).sum() / contingency_matrix.sum()
        harmonic_purity = (np.amax(f1, axis=1) * contingency_matrix.sum(axis=1)).sum() / contingency_matrix.sum()
        return (purity, inverse_purity, harmonic_purity)
    
    def topic_diversion(self, df, pred_col, pred_col_label):
        df_clustered_d=df.loc[:, [pred_col, pred_col_label]]
        df_clustered_d.drop_duplicates(inplace=True)
        all_labels=df_clustered_d[pred_col_label].to_numpy()
        topic_diversion=self.cohesion_computation(all_labels)
        return topic_diversion

            

In [6]:
INPUT_FILE='./Parsed_CCP_Actions.csv'
INPUT_COLUMN_1='ccp_long_description' 
INPUT_COLUMN_2='ccp_verbose_label'
INPUT_COLUMN_3='ccp_succinct_label'
INPUT_ID='case_no'
INPUT_TEXT='trnscr_rdndnt_crosstalk_corr_rmv_tag'
OUTPUT_FILE='./Clustered_file.csv'

EMBED_DIM = 1024 
SENTENCE_MODEL_PATH = './bge-large-en-v1.5' ## Path to Sentence Transformer Model


In [7]:
data_load=Data_load()
df=data_load.data_processing()
X1=data_load.embeddings(df)

  0%|          | 0/2025 [00:00<?, ?it/s]

summary_sentences  ["agent apologizes for the customer's frustrating experience and offers to help with the address.", "agent suggests writing a letter to the office of the president to address the customer's concerns.", "agent promises to look into the customer's concerns and provide a proper channel for their complaint."]


100%|██████████| 2025/2025 [01:45<00:00, 19.24it/s]


In [8]:
X=pd.read_pickle('./data_full_embeddings_smp.pickle')
master_clustering=Clustering(df, X)

L1_th=0.7
L2_th=0.8
min_L1=5
min_L2=2

df_clustered=master_clustering.clustering(L1_th, L2_th, min_L1, min_L2)


In [9]:
df_clustered.groupby(['L0_label', 'L1_label', 'L2_label']).size().to_frame()

0
L0_label   L1_label            L2_label                                              
Cluster: 0 Credit Limit Reason account review and confirmation                      8
                               additional payment suggestion                       19
                               agent confirms if the cardmember is calling to ...   4
                               agent informs customer about the release progra...   2
                               apologize and acknowledge inconvenience              2
...                                                                                ..
Cluster: 1 Transfer            balance inquiry transfer                            10
                               phone number collection                              2
                               supervisor connection                                4
                               transfer to billing dispute                          2
           Verify Identity     verifying customer identity and card security code   5

[237 rows x 1 columns]

In [10]:
df_clustered.to_csv(OUTPUT_FILE, index=False)